In [4]:
import pandas as pd
stack_overflow = pd.read_feather('stack_overflow.feather')
print(stack_overflow.head())

   respondent                                        main_branch hobbyist  \
0        36.0  I am not primarily a developer, but I write co...      Yes   
1        47.0                     I am a developer by profession      Yes   
2        69.0                     I am a developer by profession      Yes   
3       125.0  I am not primarily a developer, but I write co...      Yes   
4       147.0  I am not primarily a developer, but I write co...       No   

    age  age_1st_code age_first_code_cut comp_freq  comp_total  \
0  34.0          30.0              adult    Yearly     60000.0   
1  53.0          10.0              child    Yearly     58000.0   
2  25.0          12.0              child    Yearly    550000.0   
3  41.0          30.0              adult   Monthly    200000.0   
4  28.0          15.0              adult    Yearly     50000.0   

   converted_comp         country  ...          survey_length trans  \
0         77556.0  United Kingdom  ...  Appropriate in length    No  

In [3]:
# Calculate the mean of the converted_comp column
mean_comp_samp = stack_overflow['converted_comp'].mean()
print(mean_comp_samp)

119574.71738168952


In [5]:
late_shipments = pd.read_feather('late_shipments.feather')

# Print the late_shipments dataset
print(late_shipments.head())

# Calculate the proportion of late shipments
late_prop_samp = (late_shipments.late == "Yes").mean()

# Print the results
print(late_prop_samp)

        id       country managed_by  fulfill_via vendor_inco_term  \
0  36203.0       Nigeria   PMO - US  Direct Drop              EXW   
1  30998.0      Botswana   PMO - US  Direct Drop              EXW   
2  69871.0       Vietnam   PMO - US  Direct Drop              EXW   
3  17648.0  South Africa   PMO - US  Direct Drop              DDP   
4   5647.0        Uganda   PMO - US  Direct Drop              EXW   

  shipment_mode  late_delivery late product_group    sub_classification  ...  \
0           Air            1.0  Yes          HRDT              HIV test  ...   
1           Air            0.0   No          HRDT              HIV test  ...   
2           Air            0.0   No           ARV                 Adult  ...   
3         Ocean            0.0   No           ARV                 Adult  ...   
4           Air            0.0   No          HRDT  HIV test - Ancillary  ...   

  line_item_quantity line_item_value pack_price unit_price  \
0             2996.0       266644.00      

### Bootstrap sampling

In [7]:
import numpy as np

# mapping the categories to 1s and 0s
late_shipments_map = {
    'Yes': 1,
    'No': 0
}

late_shipments_mapped = late_shipments.copy()

# Convert late_shipments to a numeric column
late_shipments_mapped['late'] = late_shipments.late.map(late_shipments_map)

np.random.seed(42)
# Calculating the late shipments bootstrap distribution
late_shipments_boot_distn =[late_shipments_mapped.sample(frac=1, replace=True).late.mean() for i in range(10000)]

In [8]:
# Hypothesize that the proportion is 6%
late_prop_hyp = 0.06

# Calculate the standard error
std_error = np.std(late_shipments_boot_distn, ddof=1)

# Find z-score of late_prop_samp
z_score = (late_prop_samp - late_prop_hyp) / std_error

# Print z_score
print(z_score)

0.1322293782013778


In [9]:
from scipy.stats import norm

# Calculate the z-score of late_prop_samp
z_score = (late_prop_samp - late_prop_hyp) / std_error

# Calculate the p-value
p_value = 1 - norm.cdf(z_score, loc = 0, scale = 1)

# Print the p-value
print(p_value)

0.4474014323466098


In [10]:
# Calculate 95% confidence interval using quantile method
lower = np.quantile(late_shipments_boot_distn, 0.025)
upper = np.quantile(late_shipments_boot_distn, 0.975)

# Print the confidence interval
print((lower, upper))

(0.046, 0.076)


In [ ]:
# Calculate 95% confidence interval using t-distribution
lower = np.mean(late_shipments_boot_distn) - 1.96 * np.std(late_shipments_boot_distn, ddof=1)
upper = np.mean(late_shipments_boot_distn) + 1.96 * np.std(late_shipments_boot_distn, ddof=1)

# Print the confidence interval
print((lower, upper))